In [2]:
import numpy as np
import scanpy as sc
import pandas as pd

import sys
from os.path import join as opj

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from fg_shared import _fg_data

In [ ]:
wk10_folder = opj(_fg_data, 'SEATRAC/TB_hackday_2023/data/gideon_etal/10week')

"""Load the meta-data, cell barcodes, features (i.e., genes) and gene expression values"""
md = pd.read_csv(opj(wk10_folder, 'Updated10wk_alexandria_structured_metadata10.txt'), sep='\t', low_memory=False)
md = md.iloc[1:]

clust = pd.read_csv(opj(wk10_folder, 'all_cells_umap.txt'), sep='\t')
clust = clust.iloc[1:]
clust = clust.assign(all_X=clust['X'].astype(float),
                     all_Y=clust['Y'].astype(float))

tclust = pd.read_csv(opj(wk10_folder, 'T_Cells_UMAP.txt'), sep='\t')
tclust = tclust.iloc[1:]
tclust = tclust.assign(T_X=tclust['X'].astype(float),
                       T_Y=tclust['Y'].astype(float))

md = pd.merge(md, clust, on='NAME', how='left')
md = pd.merge(md, tclust, on='NAME', how='left')


"""Load raw counts (shape is [n_genes x n_cells] with GENES as first column, set here as index)"""
# cts1 = pd.read_csv(opj(wk10_folder, 'counts_pt_1.csv.gz')).set_index('GENE')
# cts2 = pd.read_csv(opj(wk10_folder, 'counts_pt_2.csv.gz')).set_index('GENE')

"""Load normalized counts"""
cts1 = pd.read_csv(opj(wk10_folder, 'lognormalized_pt_1.csv.gz')).set_index('GENE')
cts2 = pd.read_csv(opj(wk10_folder, 'lognormalized_pt_2.csv.gz')).set_index('GENE')
cts = pd.concat((cts1, cts2), axis=1)

In [ ]:
tot = cts.sum(axis=1)
sorti = np.argsort(tot.values)[::-1]

plot_df = cts.iloc[sorti[:20], :]
plot_df = plot_df.stack().reset_index().rename({'level_1':'cellid', 0:'value'}, axis=1)
sns.boxplot(y='GENE', x='value', data=plot_df)
plt.title('Expression of top 20 genes')

In [ ]:
gene = 'IFNG'
md = md.assign(gex=cts.loc[gene].values)

sns.histplot(md.loc[md['gex'] > 0 , 'gex'], cumulative=True)
plt.title('Cumulative distribution of cells with IFNG > 0')

In [ ]:
ss = md.sample(n=1000)
sns.scatterplot(x='all_X', y='all_Y', hue='CellTypeAnnotations', data=ss, palette='tab10')
plt.legend(bbox_to_anchor=(1, 1))
plt.scatter(x='all_X', y='all_Y', c='k', s=10, data=md.loc[md['gex'] > 0])